# Misagh Mohaghegh - 810199484

Artificial Intelligence CA1: *Search Algorithms*  
In this assignment, a problem is solved using various searching methods including BFS, IDS, and weighted A*.

Here, `INPUT` is defined so we can easily have all algorithms run the same test.

In [1]:
INPUT = 'Inputs/input_0.txt'

## Graph and States

First of all, our graph environment is described.  
Alongside the `nodes` which contain their type, sets of all nodes containing `recipes` and all `hard nodes` are kept.  
A dictionary mapping a node with a `devotee` to the recipes it requires to be satisfied is also saved in the graph's fields.

In [2]:
from enum import Flag, auto
from dataclasses import dataclass
from typing import Any, Callable


@dataclass
class Edge:
    dst: int
    weight: int


class NodeType(Flag):
    NORMAL = auto()
    DEVOTEE = auto()
    RECIPE = auto()
    HARD = auto()


@dataclass
class Node:
    adj: list[Edge]
    type: NodeType


class Graph:
    def __init__(self, n: int):
        self.nodes = [Node([], NodeType.NORMAL) for _ in range(n)]
        self.hard_nodes: set[int] = set()
        self.recipes: set[int] = set()
        self.devotee_recipes: dict[int, set[int]] = {}

    def add_edge(self, u: int, v: int, weight: int):
        self.nodes[u].adj.append(Edge(v, weight))
        self.nodes[v].adj.append(Edge(u, weight))

    def setas_hard(self, u: int):
        self.nodes[u].type &= ~NodeType.NORMAL
        self.nodes[u].type |= NodeType.HARD
        self.hard_nodes.add(u)

    def add_devotee(self, u: int, recipes: list[int]):
        self.nodes[u].type &= ~NodeType.NORMAL
        self.nodes[u].type |= NodeType.DEVOTEE
        recipes_set = set(recipes)
        self.devotee_recipes[u] = recipes_set
        self.recipes = self.recipes.union(recipes_set)
        for i in recipes:
            self.nodes[u].type &= ~NodeType.NORMAL
            self.nodes[i].type |= NodeType.RECIPE

We will now describe a state our problem solving agent is in.  
The `State` class contains fields showing the properties of a state.  
The taken path to reach a state is also stored.

Not all fields are used to check for the uniqueness of a state.  
Only the following properties are used:

- Agent's location in the graph (`pos`)
- Agent's collected recipes (`done_recipes`)
- The devotees our agent has satisfied (`done_devotees`)
- The amount of time it has spent on the current hard node (`nodetime`)

With the other fields being:

- The path up to this state (`path`)
- A map from all hard nodes to how many times they have been passed (`hard_nodes`)

The function `is_goal` checks whether a state is a goal state. This is simply checking if all devotees are satisfied.  
The function `initial_state` returns the agent's initial state in the graph.

`transition_states` returns the next state our agent can be in based on its current state.  
This checks if we are in a hard state and need to wait before we can continue.  
It then checks all adjacent nodes of our agent and calculates what state it would be in assuming it went there.  
A list of all possible next states is returned.

In [3]:
import copy


@dataclass
class State:
    pos: int
    path: list[int]
    nodetime: int
    hard_nodes: dict[int, int]
    done_recipes: set[int]
    done_devotees: set[int]

    def _tuple(self):
        return (self.pos,
                self.nodetime,
                tuple(self.done_recipes),
                tuple(self.done_devotees))

    def __hash__(self) -> int:
        return hash(self._tuple())

    def __eq__(self, other: Any) -> bool:
        return isinstance(other, State) and (
            self.pos == other.pos and
            self.nodetime == other.nodetime and
            self.done_recipes == other.done_recipes and
            self.done_devotees == other.done_devotees
        )


def is_goal(graph: Graph, state: State) -> bool:
    return len(graph.devotee_recipes) == len(state.done_devotees)


def initial_state(graph: Graph, start: int) -> State:
    return State(
        pos=start,
        path=[start],
        nodetime=0,
        hard_nodes={x: 0 for x in graph.hard_nodes},
        done_recipes=set(),
        done_devotees=set()
    )


def transition_states(graph: Graph, state: State) -> list[State]:
    if NodeType.HARD in graph.nodes[state.pos].type:
        if state.nodetime < state.hard_nodes[state.pos]:
            new_state = copy.deepcopy(state)
            new_state.nodetime += 1
            new_state.path.append(state.pos)
            return [new_state]
        else:
            state.hard_nodes[state.pos] += 1

    new_states = []
    for edge in graph.nodes[state.pos].adj:
        neigh = edge.dst

        new_state = copy.deepcopy(state)
        new_state.pos = neigh
        new_state.nodetime = 0
        new_state.path.append(neigh)

        if NodeType.RECIPE in graph.nodes[neigh].type:
            new_state.done_recipes.add(neigh)

        if NodeType.DEVOTEE in graph.nodes[neigh].type:
            if graph.devotee_recipes[neigh].issubset(new_state.done_recipes):
                new_state.done_devotees.add(neigh)

        new_states.append(new_state)

    return new_states

## Getting Input and Running the Algorithms

The following function reads the text of the input provided in `input.txt` files and returns a `graph` and our `start` node.

In [4]:
def read_input(inp: list[str]) -> tuple[Graph, int]:
    inpt = iter(inp)
    n, m = map(int, next(inpt).split())
    graph = Graph(n)
    for i in range(m):
        u, v = map(int, next(inpt).split())
        graph.add_edge(u - 1, v - 1, 0)

    _ = int(next(inpt))
    hard = map(int, next(inpt).split())
    for i in hard:
        graph.setas_hard(i - 1)

    s = int(next(inpt))
    for i in range(s):
        p, _, *recipes = map(int, next(inpt).split())
        graph.add_devotee(p - 1, [x - 1 for x in recipes])

    start = int(next(inpt)) - 1
    return (graph, start)

The `read_file` function will open an input file and read its content using `read_input`.  
This function will add the `graph` and `start` variables to the global namespace.

The `unique_consecutive` function can be used to remove duplicate adjacent items in a list, in case the printed path should not have duplicate nodes when stuck in a hard node.

In [5]:
import itertools


def read_file(filename: str):
    global graph, start

    with open(filename, 'r', encoding='utf-8') as f:
        inp = [x.rstrip('\n') for x in f]

    graph, start = read_input(inp)


def unique_consecutive(lst: list[Any]) -> list[Any]:
    return [i for i, _ in itertools.groupby(lst)]

The `run` function will run a search algorithm on the global `graph` variable.  
The search algorithm will return the goal state and the number of visited states.  
Here, the path to the goal state, its cost, and the count of visited states is printed.  
Additionally, the algorithm is ran `TEST_REPEATS` times and the average time to run the algorithm is printed.

In [6]:
import time

TEST_REPEATS = 3


def run(search_alg: Callable[[Graph, State], tuple[State, int]]):
    time_start = time.time()
    for _ in range(TEST_REPEATS):
        goal_state, num_visited = search_alg(graph, start)
    time_elapsed = (time.time() - time_start) / TEST_REPEATS

    if goal_state is None:
        print('No solution')
    else:
        print(*[x + 1 for x in goal_state.path], sep=' -> ')
        print('Path cost:', len(goal_state.path) - 1)
        print('Visited states:', num_visited)
        print('Average time:', time_elapsed)

## Breadth-First Search

The following is an implementation of **breadth-first search (BFS)**.  
The frontier list is a queue of states and our explored states are stored in a set.  
The goal check is done after the state is out of the queue.

In [7]:
def BFS(graph: Graph, start: int) -> tuple[State, int]:
    state = initial_state(graph, start)
    if is_goal(graph, state):
        return state

    frontier: list[State] = []
    visited: set[State] = set()
    frontier.append(state)
    
    while frontier:
        s = frontier.pop(0)
        if is_goal(graph, s):
            return s, len(visited)

        for next_state in transition_states(graph, s):
            if next_state in visited:
                continue
            if next_state in frontier:
                continue
            frontier.append(next_state)

        visited.add(s)

    return None, None

In [8]:
read_file(INPUT)
print(f'--- File: {INPUT} ---')
run(BFS)

--- File: Inputs/input_0.txt ---
1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Path cost: 8
Visited states: 34
Average time: 0.0019985834757486978


## Iterative-Deepening Search

The following is an implementation of **iterative-deepening search (IDS)**.  
IDS repeatedly uses *depth-limited search (DLS)* to run *depth-first search (DFS)* limited to a certain depth.  
The depth increases by one every loop until the goal state is found.

Here, DLS is implemented recursively.

In [9]:
def DLS_rec_impl(graph: Graph, state: State, visited: set[State], limit: int, num_visited: int) -> tuple[State, int]:
    if limit <= 0:
        return None, num_visited

    if is_goal(graph, state):
        return state, num_visited

    for next_state in transition_states(graph, state):
        if next_state in visited:
            continue

        visited.add(next_state)
        num_visited += 1

        res, num_visited = DLS_rec_impl(graph, next_state, visited, limit - 1, num_visited)
        visited.remove(next_state)

        if res is not None:
            return res, num_visited

    return None, num_visited


def DLS_rec(graph: Graph, start: int, depth: int) -> tuple[State, int]:
    state = initial_state(graph, start)
    visited: set[State] = set()
    visited.add(state)
    return DLS_rec_impl(graph, state, visited, depth, 0)


def IDS(graph: Graph, start: int, max_depth: int = None) -> tuple[State, int]:
    depth = 1
    all_num_visited = 0
    while True:
        if max_depth is not None and max_depth < depth:
            return None

        goal_state, num_visited = DLS_rec(graph, start, depth)
        all_num_visited += num_visited
        if goal_state is not None:
            return goal_state, all_num_visited
        depth += 1

In [10]:
read_file(INPUT)
print(f'--- File: {INPUT} ---')
run(IDS)

--- File: Inputs/input_0.txt ---
1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Path cost: 8
Visited states: 922
Average time: 0.026817480723063152


## (Weighted) A*

To implement A* search, a heuristic function is required.  
The function returns an int estimating the distance a state has until reaching the goal state.

The chosen heuristic here is the sum of recipes not yet gathered, and the devotees who have not yet gotten their food. (only unique nodes)  
This heuristic is consistent. A consistent heuristic is a heuristic that does not over-estimate the cost between 2 states.  

Take 2 states A and B, where A is a parent of B.  
Consider *h(A) = a* and *h(B) = b* which are the sum of unique nodes of not yet gathered recipes and devotees.  
To reach B from A, at least *a-b* nodes must be passed.  
Since every edge costs 1, **cost(A to B) >= a - b** and the heuristic is consistent.

In [11]:
def heuristic(state: State) -> int:
    notdone_recipes = graph.recipes.difference(state.done_recipes)
    notdone_devotees = [k for k in graph.devotee_recipes if k not in state.done_devotees]
    return len(notdone_recipes.union(notdone_devotees))

A binary heap class is implemented because Python's standard library heap does not support comparison predicates.

In [12]:
import heapq


class MinHeap:
    def __init__(self, priority_func: Callable[[Any], int] = None):
        self._data: list[tuple[int, int, Any]] = []
        self._counter = itertools.count()
        self._func = None
        if callable(priority_func):
            self._func = priority_func

    def push(self, value: Any, priority: int = None):
        if priority is None and self._func is not None:
            priority = self._func(value)

        elem = (priority, next(self._counter), value)
        heapq.heappush(self._data, elem)

    def pop(self) -> Any:
        return heapq.heappop(self._data)[-1]

    def top(self) -> Any:
        return self._data[0][-1]

    def __bool__(self):
        return bool(self._data)

    def __iter__(self):
        for i in self._data:
            yield i[-1]

The following is an implementation of **A⋆ Search**.  
This is essentially the same as BFS, but instead of a normal queue, a priority queue is used.  
The heap is sorted by a heuristic function plus the real cost to reach that state.

Weighted A* is just multiplying the heuristic value by `alpha`. So for alpha=1, it is the same as A*.  
Using an alpha value too high may not keep optimality.  
Because a consistent heuristic is always lower than the real cost, by multiplying it by an alpha, we hope to reach values closer to the real cost but this may break its consistency and therefore not return the optimal answer.

In [13]:
def AStar(graph: Graph, start: int, alpha: int = 1) -> tuple[State, int]:
    state = initial_state(graph, start)
    if is_goal(graph, state):
        return state

    def h(state: State) -> int:
        return alpha * heuristic(state) + len(state.path)

    frontier = MinHeap(h)
    visited: set[State] = set()
    frontier.push(state)

    while frontier:
        s = frontier.pop()
        if is_goal(graph, s):
            return s, len(visited)

        for next_state in transition_states(graph, s):
            if next_state in visited:
                continue
            if next_state in frontier:
                continue
            frontier.push(next_state)

        visited.add(s)

    return None, None

In [14]:
from functools import partial

read_file(INPUT)
print(f'--- File: {INPUT} ---')
run(AStar)
run(partial(AStar, alpha=1.2))
run(partial(AStar, alpha=5))

--- File: Inputs/input_0.txt ---
1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Path cost: 8
Visited states: 31
Average time: 0.0016657511393229167
1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Path cost: 8
Visited states: 24
Average time: 0.0013333956400553386
1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Path cost: 8
Visited states: 14
Average time: 0.0006658236185709635


## Comparison

Now we will run all tests on all algorithms:

In [15]:
test_cases = ['Inputs/input_0.txt', 'Inputs/input_1.txt', 'Inputs/input_2.txt']

for test in test_cases:
    read_file(test)
    print(f'=-=-=-=-=-=-=-= {test} =-=-=-=-=-=-=-=')
    print('\n--- BFS ---')
    run(BFS)
    print('\n--- IDS ---')
    run(IDS)
    print('\n--- A* ---')
    run(AStar)
    print('\n--- A* alpha=1.2 ---')
    run(partial(AStar, alpha=1.2))
    print('\n--- A* alpha=5 ---')
    run(partial(AStar, alpha=5))
    print('\n')

=-=-=-=-=-=-=-= Inputs/input_0.txt =-=-=-=-=-=-=-=

--- BFS ---
1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Path cost: 8
Visited states: 34
Average time: 0.0019981861114501953

--- IDS ---
1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Path cost: 8
Visited states: 922
Average time: 0.026388009389241535

--- A* ---
1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Path cost: 8
Visited states: 31
Average time: 0.0016672611236572266

--- A* alpha=1.2 ---
1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Path cost: 8
Visited states: 24
Average time: 0.0010000069936116536

--- A* alpha=5 ---
1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Path cost: 8
Visited states: 14
Average time: 0.0010001659393310547


=-=-=-=-=-=-=-= Inputs/input_1.txt =-=-=-=-=-=-=-=

--- BFS ---
9 -> 10 -> 9 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
Path cost: 8
Visited states: 89
Average time: 0.004666566848754883

--- IDS ---
9 -> 10 -> 9 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
Path cost: 8
Visited states: 2143
Average time: 0.06466762224833171

--- A*

In [16]:
test_cases_noids = ['Inputs/input_3.txt', 'Inputs/input_4.txt']

for test in test_cases_noids:
    read_file(test)
    print(f'=-=-=-=-=-=-=-= {test} =-=-=-=-=-=-=-=')
    print('\n--- BFS ---')
    run(BFS)
    print('\n--- A* ---')
    run(AStar)
    print('\n--- A* alpha=1.2 ---')
    run(partial(AStar, alpha=1.2))
    print('\n--- A* alpha=5 ---')
    run(partial(AStar, alpha=5))
    print('\n')

=-=-=-=-=-=-=-= Inputs/input_3.txt =-=-=-=-=-=-=-=

--- BFS ---
28 -> 19 -> 13 -> 3 -> 11 -> 24 -> 9 -> 23 -> 28 -> 23 -> 5 -> 7 -> 29
Path cost: 12
Visited states: 4595
Average time: 1.110090176264445

--- A* ---
28 -> 19 -> 13 -> 3 -> 11 -> 24 -> 9 -> 23 -> 5 -> 7 -> 29 -> 22 -> 28
Path cost: 12
Visited states: 1231
Average time: 0.5080259641011556

--- A* alpha=1.2 ---
28 -> 23 -> 9 -> 24 -> 11 -> 3 -> 13 -> 19 -> 28 -> 23 -> 5 -> 7 -> 29
Path cost: 12
Visited states: 288
Average time: 0.12134480476379395

--- A* alpha=5 ---
28 -> 19 -> 3 -> 11 -> 24 -> 9 -> 23 -> 5 -> 7 -> 29 -> 20 -> 13 -> 19 -> 28
Path cost: 13
Visited states: 15
Average time: 0.0029999415079752603


=-=-=-=-=-=-=-= Inputs/input_4.txt =-=-=-=-=-=-=-=

--- BFS ---
40 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 -> 41 -> 18 -> 1 -> 19 -> 43 -> 49 -> 47 -> 49 -> 9 -> 34 -> 25 -> 50 -> 12 -> 16
Path cost: 21
Visited states: 4850
Average time: 0.5980412165323893

--- A* ---
40 -> 42 -> 38 -> 24 -> 31 -> 45 -> 30 -> 48 ->

input_0.txt:

| Alg | Cost | Visited States | Run Time |
| :-: | :--: | :------------: | :------: |
| BFS | 8    | 34             | 0.0019   |
| IDS | 8    | 922            | 0.0263   |
| A*  | 8    | 31             | 0.0016   |
| A* Alpha=1.2 | 8 | 24       | 0.0010   |
| A* Alpha=5   | 8 | 14       | 0.0010   |

input_1.txt

| Alg | Cost | Visited States | Run Time |
| :-: | :--: | :------------: | :------: |
| BFS | 8    | 89             | 0.0046   |
| IDS | 8    | 2143           | 0.0646   |
| A*  | 8    | 48             | 0.0029   |
| A* Alpha=1.2 | 8 | 31       | 0.0020   |
| A* Alpha=5   | 9 | 14       | 0.0006   |

input_2.txt

| Alg | Cost | Visited States | Run Time |
| :-: | :--: | :------------: | :------: |
| BFS | 13   | 957            | 0.0619   |
| IDS | 13   | 37843          | 1.1186   |
| A*  | 13   | 453            | 0.0367   |
| A* Alpha=1.2 | 14 | 389     | 0.0328   |
| A* Alpha=5   | 14 | 22      | 0.0013   |

input_3.txt

| Alg | Cost | Visited States | Run Time |
| :-: | :--: | :------------: | :------: |
| BFS | 12   | 4595           | 1.1100   |
| IDS | ---  | ---            | ---      |
| A*  | 12   | 1231           | 0.5080   |
| A* Alpha=1.2 | 12 | 288     | 0.1213   |
| A* Alpha=5   | 13 | 15      | 0.0029   |

input_4.txt

| Alg | Cost | Visited States | Run Time |
| :-: | :--: | :------------: | :------: |
| BFS | 21   | 4850           | 0.5980   |
| IDS | ---  | ---            | ---      |
| A*  | 21   | 3438           | 0.5629   |
| A* Alpha=1.2 | 21 | 2768    | 0.5059   |
| A* Alpha=5   | 25 | 81      | 0.0086   |

BFS and IDS will always give us the optimal solution.  
A* with a consistent heuristic will also give the optimal solution.  
But weighted A* may not always give us the optimal solution, but if it does, it is faster than the rest of the algorithms.

As a general speed comparison: Weighted A* > A* > BFS > IDS  
Whilst IDS is the slowest algorithm, it has the lowest memory footprint.  
(IDS is polynomial while BFS and A* are exponential)